In [ ]:
using gmsh

using CairoMakie
using GeometryBasics

CairoMakie.activate!(type = "svg")

In [ ]:
gmsh.finalize()
gmsh.initialize()

# Exercise
Create the geometry shown in the figure below, parameterized with $a = 0.01\,\mathrm{m}$ and outer radius $R = 0.133\,\mathrm{m}$. Ensure that the mesh is finer at the transducer, and that the boundaries have proper labels.
<img src="exercises/semicircle_radiation_domain.png" alt="Domain" width="300"/>
The geometry is used in the paper _Isogeometric simulation of acoustic radiation_ by Victoria Hernández, et al. (https://arxiv.org/abs/2111.02546)

# Geometry Definition

## Parameters

In [ ]:
R = 0.133; # Radius of simulation domain
a = 0.01;  # Transducer aperture: 2*a

mshd_R = R / 5; # Mesh density at domain boundary
mshd_a = a / 5; # Mesh density at transducer aperture

## Gmsh

In [ ]:
gmsh.model.add("semicircle")
geo = gmsh.model.geo;

# Points
geo.addPoint(0, 0, 0, mshd_a, 1)  # Semicircle boundary midpoint

geo.addPoint(-R, 0, 0, mshd_R, 2) # Boundary left
geo.addPoint(-a, 0, 0, mshd_a, 3) # Transducer aperture left
geo.addPoint(a, 0, 0, mshd_a, 4)  # Transducer aperture right
geo.addPoint(R, 0, 0, mshd_R, 5)  # Boundary right

# Lines
geo.addLine(2, 3, 1)
geo.addLine(3, 4, 2)
geo.addLine(4, 5, 3)
geo.addCircleArc(5, 1, 2, 4)

# Surfaces
geo.addCurveLoop([1, 2, 3, 4], 1)
geo.addPlaneSurface([1], 1)

# Physics
geo.addPhysicalGroup(2, [1], 1)

geo.addPhysicalGroup(1, [2], 1)
geo.addPhysicalGroup(1, [1, 3], 2)
geo.addPhysicalGroup(1, [4], 3)

gmsh.model.setPhysicalName(2, 1, "Omega")
gmsh.model.setPhysicalName(1, 1, "Gamma_D")
gmsh.model.setPhysicalName(1, 2, "Gamma_N")
gmsh.model.setPhysicalName(1, 3, "Gamma_R")

# Generate mesh
geo.synchronize();
gmsh.model.mesh.generate(2)

gmsh.write("exercises/semicircle_radiation.msh")

In [ ]:
gmsh.fltk.run()

# Visualization

In [ ]:
# Obtain a GeometryBasics.Mesh object suitable for plotting with Makie
function get_node_mesh()
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nNode = length(node_ids)

    eType, eTag, eConn = gmsh.model.mesh.getElements(2);
    nEl = length(eTag[1])

    points   = zeros(Point2f, nNode);          # Array of vertex coordinates (x,y)
    trif     = zeros(TriangleFace{Int}, nEl);  # Array of triangular faces (n1, n2, n3)
    
    for n in 1:nNode
        points[node_ids[n]] = Point2f(node_coord[3*(n-1) + 1], node_coord[3*(n-1) + 2])
    end
    for e in 1:nEl
        trif[e] = (eConn[1][3*(e-1)+1], eConn[1][3*(e-1)+2], eConn[1][3*(e-1)+3])
    end
    
    msh = GeometryBasics.Mesh(points, trif);
    return msh;
end

In [ ]:
msh = get_node_mesh()

f, ax, pl = mesh(msh, shading = false)
wireframe!(ax, msh, color=(:black), linewidth=1, transparency=false)
ax.aspect = AxisAspect(2)
current_figure()

In [ ]:
save("exercises/semicircle_radiation.svg", f)

![Mesh](exercises/semicircle_radiation.svg)